In [ ]:
from crawl4ai import AsyncWebCrawler,CacheMode,CrawlerRunConfig,LLMConfig
import asyncio
import nest_asyncio
from pprint import pprint
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
import json

nest_asyncio.apply()

async def main():
    html = """<div class='item'>
        <h2>Item 1</h2>
        <a href='https://example.com/item1'>Link 1</a>
    </div>"""

    schema = JsonCssExtractionStrategy.generate_schema(
        html,
        llm_config=LLMConfig(
            provider="ollama/llama3.2",
            api_token=None
        )
    )
    print("schema")
    pprint(schema)

    strategy = JsonCssExtractionStrategy(schema)

    config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        extraction_strategy=strategy,
    )

    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url = f"raw://{html}",
            config = config
        )
    
    data= json.loads(result.extracted_data)
    pprint(data)
        

if __name__ == "__main__":
    asyncio.run(main())




schema
{'baseFields': [{'attribute': 'href',
                 'name': 'data_href',
                 'type': 'attribute'}],
 'baseSelector': '.item',
 'fields': [{'name': 'title', 'selector': 'h2', 'type': 'text'},
            {'attribute': 'href',
             'name': 'link',
             'selector': 'a',
             'type': 'attribute'}],
 'name': 'Item List'}


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ Raw HTML                                                                                             |
✓ | ⏱: 0.00s 

[SCRAPE].. ◆ Raw HTML                                                                                             |
✓ | ⏱: 0.00s 

[EXTRACT]. ■ Raw HTML                                                                                             |
✓ | ⏱: 0.00s 

[COMPLETE] ● Raw HTML                                                                                             |
✓ | ⏱: 0.01s 

NameError: name 'json' is not defined